# 1. 외부 Module import

>전처리에 사용할 모듈을 설치 후 import

In [2]:
from torch.utils.data import TensorDataset
import numpy as np
from numpy import concatenate
from numpy import loadtxt
import pandas as pd
from pandas import read_csv,DataFrame,concat
from pandas.plotting import register_matplotlib_converters
from math import sqrt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset,random_split,DistributedSampler,DataLoader

import random,os,copy
import pickle
from tqdm import tqdm
from collections import Counter

from sklearn.metrics import mean_absolute_error,mean_squared_error,mean_squared_log_error,r2_score,roc_auc_score,roc_curve
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,MinMaxScaler,LabelEncoder

import matplotlib.pyplot as plt
from pylab import rcParams
from matplotlib import pyplot
from matplotlib import rc
import seaborn as sns

In [3]:
print("np",np.__version__)
print("pd",pd.__version__)
print("torch",torch.__version__)

np 1.22.3
pd 1.4.2
torch 1.11.0+cu113


# 2. 모델 학습 및 테스트에 필요한 데이터 불러오기

## 2-1. 2020년 데이터 결합

>2020년 대상자 설문 데이터 불러오기

In [359]:
user_survey_2020=pd.read_csv('./user_survey_2020.csv')

In [360]:
user_survey_2020.columns

Index(['userId', 'date', 'amPm', 'startInput', 'endInput', 'sleep',
       'sleepProblem', 'dream', 'amCondition', 'amEmotion', 'pmEmotion',
       'pmStress', 'pmFatigue', 'caffeine', 'cAmount(ml)', 'alcohol',
       'aAmount(ml)'],
      dtype='object')

In [361]:
user_survey_2020['sleepProblem'].value_counts()

0.0    337
2.0     79
1.0     59
5.0     34
3.0     32
9.0     27
7.0     23
6.0     17
4.0      5
8.0      2
Name: sleepProblem, dtype: int64

In [362]:
user_survey_2020_1=user_survey_2020.drop(user_survey_2020[user_survey_2020['amPm']=='pm'].index, axis=0)
user_survey_2020_2=user_survey_2020.drop(user_survey_2020[user_survey_2020['amPm']=='am'].index, axis=0)

>전 날의 데이터를 통해 수면 관련 문항의 예측위해 날짜 조정

In [363]:
from dateutil.relativedelta import relativedelta
from datetime import datetime

In [364]:
user_survey_2020_2.insert(10,'date1','')

In [366]:
user_survey_2020_2=user_survey_2020_2.reset_index(drop=True)

In [367]:
dif = relativedelta(days=1)
for i in range(len(user_survey_2020_2)):
    user_survey_2020_2['date1'][i]=(datetime.strptime(user_survey_2020_2['date'][i],'%Y-%m-%d')-dif).strftime('%Y-%m-%d')

<ipython-input-367-5dc68c84200c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_survey_2020_2['date1'][i]=(datetime.strptime(user_survey_2020_2['date'][i],'%Y-%m-%d')-dif).strftime('%Y-%m-%d')


>신체계측 라이프로그 데이터 불러오기

In [373]:
user_sleep_2020_sorted_e4 = pd.read_csv("user_sleep_2020_sorted_e4.csv")

In [374]:
user_sleep_2020_sorted_e4=user_sleep_2020_sorted_e4[['userId', 'date','e4Bvp', 'e4Eda','e4Hr', 'e4Temp']]

In [375]:
user_survey_2020_2=pd.merge(user_survey_2020_2, user_sleep_2020_sorted_e4, left_on=['userId','date1'], right_on=['userId', 'date'], how='left')

In [376]:
user_survey_2020_2.columns

Index(['userId', 'date_x', 'amPm', 'startInput', 'endInput', 'sleep',
       'sleepProblem', 'dream', 'amCondition', 'amEmotion', 'date1',
       'pmEmotion', 'pmStress', 'pmFatigue', 'caffeine', 'cAmount(ml)',
       'alcohol', 'aAmount(ml)', 'date_y', 'e4Bvp', 'e4Eda', 'e4Hr', 'e4Temp'],
      dtype='object')

In [377]:
user_survey_2020_2=user_survey_2020_2.drop(columns=['date_x', 'date_y'])

>2020년 수면과 운동이 결합된 데이터 불러오기

>운동 거리 추출

In [378]:
user_sleep_2020_exer1 = pd.read_csv("user_sleep_2020_exer1.csv")

In [379]:
user_sleep_2020_exer1=user_sleep_2020_exer1[['userId', 'date','gps_activity_distance']]

>설문과 운동 거리 데이터 머지

In [380]:
user_survey_2020_2=pd.merge(user_survey_2020_2, user_sleep_2020_exer1, left_on=['userId','date1'], right_on=['userId', 'date'], how='left')

In [381]:
user_survey_2020_2=user_survey_2020_2.drop(columns=['date'])

In [382]:
user = pd.merge(user, user_survey_2020_2, left_on =['userId','date'], right_on=['userId', 'date1'], how = 'left' )

In [384]:
user=user.drop(columns=['amPm','startInput_y', 'endInput_y', 'sleep_y', 'sleepProblem_y', 'dream_y',
       'amCondition_y', 'amEmotion_y'])

In [220]:
#0 만족함 1, 만족못함
outcome2=user['sleep_x'].apply(lambda x: 1 if x==1 or x==2 else 0)

In [ ]:
#0  수면 문제없음, 1 있음
outcome=user['sleepProblem_x'].apply(lambda x: 0 if x==0 else 1)

In [ ]:
#0 꿈 안꿈, 1 꿈 꿈
outcome1=user['dream_x'].apply(lambda x: 0 if x==4 else 1)

In [ ]:
#0 만족함 1, 만족못함
outcome3=user['amCondition_x'].apply(lambda x: 1 if x==1 or x==2 else 0)

In [ ]:
#0 만족함 1, 만족못함
outcome4=user['amEmotion_x'].apply(lambda x: 1 if x==1 or x==2 else 0)

>2020년 대상자 기본정보 추출 및 머지

In [388]:
user_info_2020=pd.read_csv('./user_info_2020.csv')

In [389]:
user = pd.merge(user, user_info_2020, left_on =['userId'], right_on=['userId'], how = 'left' )

In [390]:
user.columns

Index(['userId', 'timezone', 'date', 'startDt_x', 'endDt_x', 'lastUpdate',
       'wakeupduration', 'lightsleepduration', 'deepsleepduration',
       'wakeupcount', 'durationtosleep', 'remsleepduration',
       'durationtowakeup', 'hr_average', 'hr_min', 'hr_max', 'rr_average',
       'rr_min', 'rr_max', 'breathing_disturbances_intensity', 'snoring',
       'snoringepisodecount', 'sleep_score', 'count', 'total', 'rank', 'walk',
       'exer', 'startInput_x', 'endInput_x', 'sleep_x', 'sleepProblem_x',
       'dream_x', 'amCondition_x', 'amEmotion_x', 'date1', 'pmEmotion',
       'pmStress', 'pmFatigue', 'caffeine', 'cAmount(ml)', 'alcohol',
       'aAmount(ml)', 'e4Bvp', 'e4Eda', 'e4Hr', 'e4Temp',
       'gps_activity_distance', 'gender', 'age', 'handed', 'height', 'weight',
       'startDt_y', 'endDt_y'],
      dtype='object')

In [391]:
user=user.drop(columns=['startDt_y', 'endDt_y'])

In [392]:
user['alcohol'].isna().sum()

438

In [394]:
user.columns

Index(['userId', 'timezone', 'date', 'startDt_x', 'endDt_x', 'lastUpdate',
       'wakeupduration', 'lightsleepduration', 'deepsleepduration',
       'wakeupcount', 'durationtosleep', 'remsleepduration',
       'durationtowakeup', 'hr_average', 'hr_min', 'hr_max', 'rr_average',
       'rr_min', 'rr_max', 'breathing_disturbances_intensity', 'snoring',
       'snoringepisodecount', 'sleep_score', 'count', 'total', 'rank', 'walk',
       'exer', 'startInput_x', 'endInput_x', 'sleep_x', 'sleepProblem_x',
       'dream_x', 'amCondition_x', 'amEmotion_x', 'date1', 'pmEmotion',
       'pmStress', 'pmFatigue', 'caffeine', 'cAmount(ml)', 'alcohol',
       'aAmount(ml)', 'e4Bvp', 'e4Eda', 'e4Hr', 'e4Temp',
       'gps_activity_distance', 'gender', 'age', 'handed', 'height', 'weight'],
      dtype='object')

In [228]:
user_sleep_2020_exer1

,userId,date,gps_activity_distance
0,user01,2020-08-31,NaN
1,user01,2020-09-01,NaN
2,user01,2020-09-06,NaN
3,user01,2020-09-08,NaN
4,user01,2020-09-09,NaN
...,...,...,...
485,user30,2020-09-22,NaN
486,user30,2020-09-23,NaN
487,user30,2020-09-24,NaN
488,user30,2020-09-25,NaN


## 2-2. 2019년 데이터 결합

>2019년 수면과 운동이 결합된 데이터 불러오기

In [410]:
user_sleep_2019_exer1=pd.read_csv('./user_sleep_2019_exer1.csv')

In [411]:
user_sleep_2019_exer1=user_sleep_2019_exer1.drop(columns=['Unnamed: 0.1','Unnamed: 0'])

In [412]:
user_survey_2019_2018=pd.read_csv('./user_survey_2019_2018.csv')

In [413]:
user_survey_2019_2018.columns

Index(['userId', 'amPm', 'inputDt', 'sleep', 'sleepProblem', 'dream',
       'amCondition', 'amEmotion', 'pmEmotion', 'pmStress', 'ifUnusual',
       'breakfast', 'lunch', 'dinner', 'lateSnack', 'amCaffeine',
       'amCaffAmount', 'pmCaffeine', 'pmCaffAmount', 'alcohol', 'aAmount'],
      dtype='object')

In [414]:
user_sleep_2019_exer1

,startDt,endDt,device,sleep_score,total_sleep_time,time_in_bed,waso,wakeupcount,aal,movement_index,...,sleep_frag_index,startdt1,enddt1,count,date,userId,walk,exer,userName,gps_activity_distance
0,16DEC19:10:41:00,16DEC19:20:56:00,Actigraph,97.89,36120,36900,780,2,6.50,12.060,...,12.060,20191216,20191216,2,2019-12-15,user02,0,0,NaN,NaN
1,16DEC19:00:23:00,16DEC19:09:23:00,Actigraph,82.44,26760,32460,5700,3,31.67,6.412,...,31.412,20191216,20191216,2,2019-12-15,user02,0,0,NaN,NaN
2,05DEC19:02:21:00,05DEC19:07:05:00,Actigraph,96.48,16440,17040,600,8,1.25,4.930,...,4.930,20191205,20191205,3,2019-12-04,user04,50,0,user04,0.0
3,05DEC19:08:10:00,05DEC19:17:29:00,Actigraph,98.03,32880,33540,660,4,2.75,1.431,...,1.431,20191205,20191205,3,2019-12-04,user04,50,0,user04,0.0
4,05DEC19:18:16:00,06DEC19:01:20:00,Actigraph,99.53,25320,25440,120,2,1.00,2.358,...,2.358,20191205,20191206,3,2019-12-05,user04,14,0,user04,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
334,10FEB20:11:14:00,11FEB20:02:49:00,Actigraph,100.00,56100,56100,0,0,0.00,0.214,...,0.214,20200210,20200211,2,2020-02-10,user20,34,0,user20,0.0
335,11FEB20:06:44:00,11FEB20:11:13:00,Actigraph,87.36,14100,16140,2040,16,2.13,10.037,...,27.684,20200211,20200211,2,2020-02-10,user20,34,0,user20,0.0
336,11FEB20:11:24:00,12FEB20:05:03:00,Actigraph,100.00,63540,63540,0,0,0.00,0.378,...,0.378,20200211,20200212,2,2020-02-11,user20,132,0,user20,0.0
337,12FEB20:07:22:00,13FEB20:04:00:00,Actigraph,96.45,71640,74280,2640,19,2.32,3.150,...,3.150,20200212,20200213,1,2020-02-12,user20,156,0,user20,0.0


>2019년 대상자 설문 데이터 불러오기

In [415]:
user_survey_2019=pd.read_csv('./user_survey_2019.csv')

In [416]:
user_survey_2019['userId1']=""
for i in range(len(user_survey_2019)):
    if user_survey_2019['userId'][i]==101:
        user_survey_2019['userId1'][i]='user01'
    elif user_survey_2019['userId'][i]==102:
        user_survey_2019['userId1'][i]='user02'
    elif user_survey_2019['userId'][i]==103:
        user_survey_2019['userId1'][i]='user03'
    elif user_survey_2019['userId'][i]==104:
        user_survey_2019['userId1'][i]='user04'
    elif user_survey_2019['userId'][i]==105:
        user_survey_2019['userId1'][i]='user05'
    elif user_survey_2019['userId'][i]==106:
        user_survey_2019['userId1'][i]='user06'
    elif user_survey_2019['userId'][i]==107:
        user_survey_2019['userId1'][i]='user07'
    elif user_survey_2019['userId'][i]==108:
        user_survey_2019['userId1'][i]='user08'
    elif user_survey_2019['userId'][i]==109:
        user_survey_2019['userId1'][i]='user09'
    elif user_survey_2019['userId'][i]==110:
        user_survey_2019['userId1'][i]='user10'
    elif user_survey_2019['userId'][i]==111:
        user_survey_2019['userId1'][i]='user11'
    elif user_survey_2019['userId'][i]==112:
        user_survey_2019['userId1'][i]='user12'
    elif user_survey_2019['userId'][i]==113:
        user_survey_2019['userId1'][i]='user13'
    elif user_survey_2019['userId'][i]==114:
        user_survey_2019['userId1'][i]='user14'
    elif user_survey_2019['userId'][i]==115:
        user_survey_2019['userId1'][i]='user15'
    elif user_survey_2019['userId'][i]==116:
        user_survey_2019['userId1'][i]='user16'
    elif user_survey_2019['userId'][i]==117:
        user_survey_2019['userId1'][i]='user17'
    elif user_survey_2019['userId'][i]==118:
        user_survey_2019['userId1'][i]='user18'
    elif user_survey_2019['userId'][i]==119:
        user_survey_2019['userId1'][i]='user19'
    elif user_survey_2019['userId'][i]==120:
        user_survey_2019['userId1'][i]='user20'

<ipython-input-416-eed064f3c41b>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_survey_2019['userId1'][i]='user17'
<ipython-input-416-eed064f3c41b>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_survey_2019['userId1'][i]='user05'
<ipython-input-416-eed064f3c41b>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_survey_2019['userId1'][i]='user04'
<ipython-input-416-eed064f3c41b>:42: SettingWithCopyWarning: 
A value 

In [417]:
user_survey_2019_1=user_survey_2019.drop(user_survey_2019[user_survey_2019['amPm']=='pm'].index, axis=0)
user_survey_2019_2=user_survey_2019.drop(user_survey_2019[user_survey_2019['amPm']=='am'].index, axis=0)

In [418]:
user_survey_2019_1=user_survey_2019_1.reset_index(drop=True)
user_survey_2019_2=user_survey_2019_2.reset_index(drop=True)

In [419]:
user_survey_2019_2['inputdt1']

0      20200118
1      20200119
2      20200120
3      20200121
4      20200122
         ...   
279    20200131
280    20200201
281    20200202
282    20200203
283    20200204
Name: inputdt1, Length: 284, dtype: int64

>2019년에 등록된 대상자의 전날 결과를 통해 당일의 수면 관련 문항 예측을 위해 날짜 조정

In [420]:
from datetime import datetime, timedelta
dif = relativedelta(days=1)
for i in range(len(user_survey_2019_2)):
    user_survey_2019_2['startdt'][i]=(datetime.strptime(str(user_survey_2019_2['inputdt1'][i]),'%Y%m%d')-dif).strftime('%Y-%m-%d')

<ipython-input-420-f158ec152091>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_survey_2019_2['startdt'][i]=(datetime.strptime(str(user_survey_2019_2['inputdt1'][i]),'%Y%m%d')-dif).strftime('%Y-%m-%d')


In [421]:
user_survey_2019_2=user_survey_2019_2.rename(columns={'startdt':'date1'})

In [422]:
for i in range(len(user_survey_2019_1)):
    user_survey_2019_1['inputdt1'][i]=datetime.strptime(str(user_survey_2019_1['inputdt1'][i]),'%Y%m%d').strftime('%Y-%m-%d')

<ipython-input-422-471df3ebddbb>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_survey_2019_1['inputdt1'][i]=datetime.strptime(str(user_survey_2019_1['inputdt1'][i]),'%Y%m%d').strftime('%Y-%m-%d')


In [423]:
user_survey_2019_1=user_survey_2019_1.drop(columns=['userId'])
user_survey_2019_1=user_survey_2019_1.rename(columns={'userId1':'userId'})

In [424]:
user_survey_2019_1=user_survey_2019_1.rename(columns={'inputdt1':'date1'})

In [425]:
user_sleep_2019_exer1=user_sleep_2019_exer1.drop(columns=['userName'])

In [426]:
user_survey_2019_1=user_survey_2019_1[['sleep', 'sleepProblem','dream','amCondition','amEmotion','date1','userId']]

>2019년 대상자의 데이터에 설문 데이터 머지

In [427]:
user1 = pd.merge(user_sleep_2019_exer1, user_survey_2019_1, left_on =['userId','date'], right_on=['userId', 'date1'], how = 'left' )

In [428]:
user_survey_2019_2.columns

Index(['userId', 'amPm', 'inputDt', 'sleep', 'sleepProblem', 'dream',
       'amCondition', 'amEmotion', 'pmEmotion', 'pmStress', 'ifUnusual',
       'breakfast', 'lunch', 'dinner', 'lateSnack', 'amCaffeine',
       'amCaffAmount', 'pmCaffeine', 'pmCaffAmount', 'alcohol', 'aAmount',
       'startdt1', 'date1', 'enddt1', 'enddt', 'inputdt1', 'userId1'],
      dtype='object')

In [429]:
user_survey_2019_2=user_survey_2019_2[['date1','userId1', 'pmEmotion', 'pmStress','amCaffeine',
       'amCaffAmount', 'pmCaffeine', 'pmCaffAmount', 'alcohol', 'aAmount']]

In [430]:
user_survey_2019_2=user_survey_2019_2.rename(columns={'userId1':'userId'})

In [431]:
user_survey_2019_2['amCaffeine']=user_survey_2019_2['amCaffeine'].fillna(0)
user_survey_2019_2['pmCaffeine']=user_survey_2019_2['pmCaffeine'].fillna(0)
user_survey_2019_2['amCaffAmount']=user_survey_2019_2['amCaffAmount'].fillna(0)
user_survey_2019_2['pmCaffAmount']=user_survey_2019_2['pmCaffAmount'].fillna(0)
user_survey_2019_2['alcohol']=user_survey_2019_2['alcohol'].fillna(0)
user_survey_2019_2['aAmount']=user_survey_2019_2['aAmount'].fillna(0)

In [432]:
user_survey_2019_2['caffeine']=''
user_survey_2019_2['cAmount(ml)']=''
user_survey_2019_2['aAmount(ml)']=''

In [433]:
user_survey_2019_2['amCaffeine']=user_survey_2019_2['amCaffeine'].apply(lambda x: 0 if x==0 else 1)
user_survey_2019_2['pmCaffeine']=user_survey_2019_2['pmCaffeine'].apply(lambda x: 0 if x==0 else 1)
user_survey_2019_2['alcohol']=user_survey_2019_2['alcohol'].apply(lambda x: 0 if x==0 else 1)

In [434]:
user_survey_2019_2['caffeine']=user_survey_2019_2['amCaffeine']+user_survey_2019_2['pmCaffeine']

In [435]:
user_survey_2019_2['caffeine']=user_survey_2019_2['caffeine'].apply(lambda x: 0 if x==0 else 1)

In [436]:
user_survey_2019_2['cAmount(ml)']=user_survey_2019_2['amCaffAmount']+user_survey_2019_2['pmCaffAmount']

In [437]:
user_survey_2019_2['aAmount(ml)']=user_survey_2019_2['aAmount']

In [438]:
user_survey_2019_2=user_survey_2019_2[['date1','userId', 'pmEmotion', 'pmStress','caffeine',
       'cAmount(ml)', 'alcohol', 'aAmount(ml)']]

In [439]:
user1=user1.drop(columns='date1')

In [440]:
user1 = pd.merge(user1, user_survey_2019_2, left_on =['userId','date'], right_on=['userId', 'date1'], how = 'left' )

>신체계측 데이터 추출

In [441]:
user_sleep_2019_2018_sorted_e4=pd.read_csv("user_sleep_2019_2018_sorted_e4.csv")

In [442]:
user_sleep_2019_2018_sorted_e4=user_sleep_2019_2018_sorted_e4[['userId','startDt','e4Bvp','e4Eda','e4Hr','e4Temp']]

In [443]:
user_sleep_2019_e4=user_sleep_2019_2018_sorted_e4.loc[user_sleep_2019_2018_sorted_e4['userId']>100,:]

In [444]:
user_sleep_2019_e4=user_sleep_2019_e4.reset_index(drop=True)

In [445]:
user_sleep_2019_e4['userId1']=""
for i in range(len(user_sleep_2019_e4)):
    if user_sleep_2019_e4['userId'][i]==101:
        user_sleep_2019_e4['userId1'][i]='user01'
    elif user_sleep_2019_e4['userId'][i]==102:
        user_sleep_2019_e4['userId1'][i]='user02'
    elif user_sleep_2019_e4['userId'][i]==103:
        user_sleep_2019_e4['userId1'][i]='user03'
    elif user_sleep_2019_e4['userId'][i]==104:
        user_sleep_2019_e4['userId1'][i]='user04'
    elif user_sleep_2019_e4['userId'][i]==105:
        user_sleep_2019_e4['userId1'][i]='user05'
    elif user_sleep_2019_e4['userId'][i]==106:
        user_sleep_2019_e4['userId1'][i]='user06'
    elif user_sleep_2019_e4['userId'][i]==107:
        user_sleep_2019_e4['userId1'][i]='user07'
    elif user_sleep_2019_e4['userId'][i]==108:
        user_sleep_2019_e4['userId1'][i]='user08'
    elif user_sleep_2019_e4['userId'][i]==109:
        user_sleep_2019_e4['userId1'][i]='user09'
    elif user_sleep_2019_e4['userId'][i]==110:
        user_sleep_2019_e4['userId1'][i]='user10'
    elif user_sleep_2019_e4['userId'][i]==111:
        user_sleep_2019_e4['userId1'][i]='user11'
    elif user_sleep_2019_e4['userId'][i]==112:
        user_sleep_2019_e4['userId1'][i]='user12'
    elif user_sleep_2019_e4['userId'][i]==113:
        user_sleep_2019_e4['userId1'][i]='user13'
    elif user_sleep_2019_e4['userId'][i]==114:
        user_sleep_2019_e4['userId1'][i]='user14'
    elif user_sleep_2019_e4['userId'][i]==115:
        user_sleep_2019_e4['userId1'][i]='user15'
    elif user_sleep_2019_e4['userId'][i]==116:
        user_sleep_2019_e4['userId1'][i]='user16'
    elif user_sleep_2019_e4['userId'][i]==117:
        user_sleep_2019_e4['userId1'][i]='user17'
    elif user_sleep_2019_e4['userId'][i]==118:
        user_sleep_2019_e4['userId1'][i]='user18'
    elif user_sleep_2019_e4['userId'][i]==119:
        user_sleep_2019_e4['userId1'][i]='user19'
    elif user_sleep_2019_e4['userId'][i]==120:
        user_sleep_2019_e4['userId1'][i]='user20'

<ipython-input-445-06200794e73a>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_sleep_2019_e4['userId1'][i]='user02'
<ipython-input-445-06200794e73a>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_sleep_2019_e4['userId1'][i]='user04'
<ipython-input-445-06200794e73a>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_sleep_2019_e4['userId1'][i]='user05'
<ipython-input-445-06200794e73a>:14: SettingWithCopyWarning: 
A v

In [446]:
user_sleep_2019_e4=user_sleep_2019_e4.drop(columns=['userId'])
user_sleep_2019_e4=user_sleep_2019_e4.rename(columns={'userId1':'userId'})

In [447]:
user_sleep_2019_e4=user_sleep_2019_e4.rename(columns={'startDt':'date'})

In [448]:
user_sleep_2019_e4

,date,e4Bvp,e4Eda,e4Hr,e4Temp,userId
0,2019-12-16 0:23,-0.000070,0.012726,90.082467,32.928060,user02
1,2019-12-16 10:41,-0.000070,0.012726,90.082467,32.928060,user02
2,2019-12-05 18:16,0.021180,2.346086,71.768512,34.364740,user04
3,2019-12-05 2:21,-0.007682,0.388813,77.905570,33.412815,user04
4,2019-12-05 8:10,-0.007682,0.388813,77.905570,33.412815,user04
...,...,...,...,...,...,...
403,2020-02-10 3:29,-0.000073,0.262322,64.862948,33.655236,user20
404,2020-02-11 11:24,-0.005079,0.416336,99.479818,32.241794,user20
405,2020-02-11 6:44,-0.001991,0.551994,93.185853,32.848619,user20
406,2020-02-12 7:22,NaN,NaN,NaN,NaN,user20


In [449]:
user_sleep_2019_e4['date1']=''
for i in range(len(user_sleep_2019_e4)):
    user_sleep_2019_e4['date1'][i]=datetime.strptime(str(user_sleep_2019_e4['date'][i]),'%Y-%m-%d %H:%M').strftime('%Y-%m-%d')

<ipython-input-449-b33a00e87a95>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_sleep_2019_e4['date1'][i]=datetime.strptime(str(user_sleep_2019_e4['date'][i]),'%Y-%m-%d %H:%M').strftime('%Y-%m-%d')


In [450]:
user_sleep_2019_e4=user_sleep_2019_e4.drop(columns='date')

In [451]:
user1.columns

Index(['startDt', 'endDt', 'device', 'sleep_score', 'total_sleep_time',
       'time_in_bed', 'waso', 'wakeupcount', 'aal', 'movement_index',
       'fragmentation_index', 'sleep_frag_index', 'startdt1', 'enddt1',
       'count', 'date', 'userId', 'walk', 'exer', 'gps_activity_distance',
       'sleep', 'sleepProblem', 'dream', 'amCondition', 'amEmotion', 'date1',
       'pmEmotion', 'pmStress', 'caffeine', 'cAmount(ml)', 'alcohol',
       'aAmount(ml)'],
      dtype='object')

>2019년 데이터와 신체계측 데이터 머지

In [452]:
user1 = pd.merge(user1, user_sleep_2019_e4, left_on =['userId','date'], right_on=['userId', 'date1'], how = 'left' )

>2019년 대상자 정보 추출

In [453]:
user_info_2019_2018 = pd.read_csv('user_info_2019_2018_updated.csv')

In [454]:
user_info_2019=user_info_2019_2018.iloc[:20,:]

In [455]:
user_info_2019['userId'][0]

'101'

In [456]:
user_info_2019['userId1']=""
for i in range(len(user_info_2019)):
    if user_info_2019['userId'][i]=='101':
        user_info_2019['userId1'][i]='user01'
    elif user_info_2019['userId'][i]=='102':
        user_info_2019['userId1'][i]='user02'
    elif user_info_2019['userId'][i]=='103':
        user_info_2019['userId1'][i]='user03'
    elif user_info_2019['userId'][i]=='104':
        user_info_2019['userId1'][i]='user04'
    elif user_info_2019['userId'][i]=='105':
        user_info_2019['userId1'][i]='user05'
    elif user_info_2019['userId'][i]=='106':
        user_info_2019['userId1'][i]='user06'
    elif user_info_2019['userId'][i]=='107':
        user_info_2019['userId1'][i]='user07'
    elif user_info_2019['userId'][i]=='108':
        user_info_2019['userId1'][i]='user08'
    elif user_info_2019['userId'][i]=='109':
        user_info_2019['userId1'][i]='user09'
    elif user_info_2019['userId'][i]=='110':
        user_info_2019['userId1'][i]='user10'
    elif user_info_2019['userId'][i]=='111':
        user_info_2019['userId1'][i]='user11'
    elif user_info_2019['userId'][i]=='112':
        user_info_2019['userId1'][i]='user12'
    elif user_info_2019['userId'][i]=='113':
        user_info_2019['userId1'][i]='user13'
    elif user_info_2019['userId'][i]=='114':
        user_info_2019['userId1'][i]='user14'
    elif user_info_2019['userId'][i]=='115':
        user_info_2019['userId1'][i]='user15'
    elif user_info_2019['userId'][i]=='116':
        user_info_2019['userId1'][i]='user16'
    elif user_info_2019['userId'][i]=='117':
        user_info_2019['userId1'][i]='user17'
    elif user_info_2019['userId'][i]=='118':
        user_info_2019['userId1'][i]='user18'
    elif user_info_2019['userId'][i]=='119':
        user_info_2019['userId1'][i]='user19'
    elif user_info_2019['userId'][i]=='120':
        user_info_2019['userId1'][i]='user20'

<ipython-input-456-99083a02908b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_info_2019['userId1']=""
<ipython-input-456-99083a02908b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_info_2019['userId1'][i]='user01'
<ipython-input-456-99083a02908b>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_info_2019['userId1'][i]='user02'
<ipython-input-456-99083a02908b

In [457]:
user_info_2019=user_info_2019.drop(columns='userId')
user_info_2019=user_info_2019.rename(columns={'userId1':'userId'})

In [458]:
user_info_2019=user_info_2019.drop(columns='endDt')

In [459]:
user_info_2019=user_info_2019.drop(columns='startDt')

>2019년 데이터와 개인정보 데이터 머지

In [460]:
user1 = pd.merge(user1, user_info_2019, left_on =['userId'], right_on=['userId'], how = 'left' )

In [461]:
user1.columns

Index(['startDt', 'endDt', 'device', 'sleep_score', 'total_sleep_time',
       'time_in_bed', 'waso', 'wakeupcount', 'aal', 'movement_index',
       'fragmentation_index', 'sleep_frag_index', 'startdt1', 'enddt1',
       'count', 'date', 'userId', 'walk', 'exer', 'gps_activity_distance',
       'sleep', 'sleepProblem', 'dream', 'amCondition', 'amEmotion', 'date1_x',
       'pmEmotion', 'pmStress', 'caffeine', 'cAmount(ml)', 'alcohol',
       'aAmount(ml)', 'e4Bvp', 'e4Eda', 'e4Hr', 'e4Temp', 'date1_y', 'gender',
       'age', 'height', 'weight'],
      dtype='object')

## 2-3. 2018년 데이터 결합

>데이터 전처리는 2019년, 2020년과 동일함.

>2018년 수면과 운동이 결합된 데이터 불러오기

In [281]:
user_survey_2018=pd.read_csv('./user_survey_2018.csv')

In [282]:
user_survey_2018['userId1']=""
for i in range(len(user_survey_2018)):
    if user_survey_2018['userId'][i]==1:
        user_survey_2018['userId1'][i]='user01'
    elif user_survey_2018['userId'][i]==2:
        user_survey_2018['userId1'][i]='user02'
    elif user_survey_2018['userId'][i]==3:
        user_survey_2018['userId1'][i]='user03'
    elif user_survey_2018['userId'][i]==4:
        user_survey_2018['userId1'][i]='user04'
    elif user_survey_2018['userId'][i]==5:
        user_survey_2018['userId1'][i]='user05'
    elif user_survey_2018['userId'][i]==6:
        user_survey_2018['userId1'][i]='user06'
    elif user_survey_2018['userId'][i]==7:
        user_survey_2018['userId1'][i]='user07'
    elif user_survey_2018['userId'][i]==8:
        user_survey_2018['userId1'][i]='user08'
    elif user_survey_2018['userId'][i]==9:
        user_survey_2018['userId1'][i]='user09'
    elif user_survey_2018['userId'][i]==10:
        user_survey_2018['userId1'][i]='user10'
    elif user_survey_2018['userId'][i]==11:
        user_survey_2018['userId1'][i]='user11'
    elif user_survey_2018['userId'][i]==12:
        user_survey_2018['userId1'][i]='user12'
    elif user_survey_2018['userId'][i]==13:
        user_survey_2018['userId1'][i]='user13'
    elif user_survey_2018['userId'][i]==14:
        user_survey_2018['userId1'][i]='user14'
    elif user_survey_2018['userId'][i]==15:
        user_survey_2018['userId1'][i]='user15'
    elif user_survey_2018['userId'][i]==16:
        user_survey_2018['userId1'][i]='user16'
    elif user_survey_2018['userId'][i]==17:
        user_survey_2018['userId1'][i]='user17'
    elif user_survey_2018['userId'][i]==18:
        user_survey_2018['userId1'][i]='user18'
    elif user_survey_2018['userId'][i]==19:
        user_survey_2018['userId1'][i]='user19'
    elif user_survey_2018['userId'][i]==20:
        user_survey_2018['userId1'][i]='user20'
    elif user_survey_2018['userId'][i]==21:
        user_survey_2018['userId1'][i]='user21'
    elif user_survey_2018['userId'][i]==22:
        user_survey_2018['userId1'][i]='user22'
    elif user_survey_2018['userId'][i]==23:
        user_survey_2018['userId1'][i]='user23'
    elif user_survey_2018['userId'][i]==24:
        user_survey_2018['userId1'][i]='user24'
    elif user_survey_2018['userId'][i]==25:
        user_survey_2018['userId1'][i]='user25'
    elif user_survey_2018['userId'][i]==26:
        user_survey_2018['userId1'][i]='user26'
    elif user_survey_2018['userId'][i]==27:
        user_survey_2018['userId1'][i]='user27'
    elif user_survey_2018['userId'][i]==28:
        user_survey_2018['userId1'][i]='user28'
    elif user_survey_2018['userId'][i]==29:
        user_survey_2018['userId1'][i]='user29'
    elif user_survey_2018['userId'][i]==30:
        user_survey_2018['userId1'][i]='user30'

<ipython-input-282-56ebbdef0610>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_survey_2018['userId1'][i]='user17'
<ipython-input-282-56ebbdef0610>:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_survey_2018['userId1'][i]='user24'
<ipython-input-282-56ebbdef0610>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_survey_2018['userId1'][i]='user04'
<ipython-input-282-56ebbdef0610>:52: SettingWithCopyWarning: 
A value 

In [283]:
user_survey_2018_1=user_survey_2018.drop(user_survey_2018[user_survey_2018['amPm']=='pm'].index, axis=0)
user_survey_2018_2=user_survey_2018.drop(user_survey_2018[user_survey_2018['amPm']=='am'].index, axis=0)

In [284]:
user_survey_2018_1=user_survey_2018_1.reset_index(drop=True)
user_survey_2018_2=user_survey_2018_2.reset_index(drop=True)

In [285]:
user_survey_2018_2['inputdt1']

0      20181031
1      20181101
2      20181103
3      20181105
4      20181106
         ...   
315    20181122
316    20181124
317    20181125
318    20181126
319    20181128
Name: inputdt1, Length: 320, dtype: int64

In [286]:
from datetime import datetime, timedelta
dif = relativedelta(days=1)
for i in range(len(user_survey_2018_2)):
    user_survey_2018_2['startdt'][i]=(datetime.strptime(str(user_survey_2018_2['inputdt1'][i]),'%Y%m%d')-dif).strftime('%Y-%m-%d')

<ipython-input-286-8d436ef972d7>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_survey_2018_2['startdt'][i]=(datetime.strptime(str(user_survey_2018_2['inputdt1'][i]),'%Y%m%d')-dif).strftime('%Y-%m-%d')


In [287]:
user_survey_2018_2=user_survey_2018_2.rename(columns={'startdt':'date1'})

In [288]:
for i in range(len(user_survey_2018_1)):
    user_survey_2018_1['inputdt1'][i]=datetime.strptime(str(user_survey_2018_1['inputdt1'][i]),'%Y%m%d').strftime('%Y-%m-%d')

<ipython-input-288-e83f34594ee0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_survey_2018_1['inputdt1'][i]=datetime.strptime(str(user_survey_2018_1['inputdt1'][i]),'%Y%m%d').strftime('%Y-%m-%d')


In [289]:
user_survey_2018_1=user_survey_2018_1.drop(columns=['userId'])
user_survey_2018_1=user_survey_2018_1.rename(columns={'userId1':'userId'})

In [290]:
user_survey_2018_1=user_survey_2018_1.rename(columns={'inputdt1':'date1'})

In [291]:
user_sleep_2018_exer1=pd.read_csv('user_sleep_2018_exer1.csv')

In [292]:
user_sleep_2018_exer1=user_sleep_2018_exer1.drop(columns=['userName'])

In [293]:
user_sleep_2018_exer1=user_sleep_2018_exer1.drop(columns=['Unnamed: 0.1','Unnamed: 0'])

In [294]:
user_survey_2018_1=user_survey_2018_1[['sleep', 'sleepProblem','dream','amCondition','amEmotion','date1','userId']]

In [295]:
user2 = pd.merge(user_sleep_2018_exer1, user_survey_2018_1, left_on =['userId','date'], right_on=['userId', 'date1'], how = 'left' )

In [296]:
user_survey_2018_2.columns

Index(['userId', 'amPm', 'inputDt', 'sleep', 'sleepProblem', 'dream',
       'amCondition', 'amEmotion', 'pmEmotion', 'pmStress', 'ifUnusual',
       'breakfast', 'lunch', 'dinner', 'lateSnack', 'amCaffeine',
       'amCaffAmount', 'pmCaffeine', 'pmCaffAmount', 'alcohol', 'aAmount',
       'startdt1', 'date1', 'enddt1', 'enddt', 'inputdt1', 'userId1'],
      dtype='object')

In [297]:
user_survey_2018_2=user_survey_2018_2[['date1','userId1', 'pmEmotion', 'pmStress','amCaffeine',
       'amCaffAmount', 'pmCaffeine', 'pmCaffAmount', 'alcohol', 'aAmount']]

In [298]:
user_survey_2018_2=user_survey_2018_2.rename(columns={'userId1':'userId'})

In [299]:
user_survey_2018_2['amCaffeine']=user_survey_2018_2['amCaffeine'].fillna(0)
user_survey_2018_2['pmCaffeine']=user_survey_2018_2['pmCaffeine'].fillna(0)
user_survey_2018_2['amCaffAmount']=user_survey_2018_2['amCaffAmount'].fillna(0)
user_survey_2018_2['pmCaffAmount']=user_survey_2018_2['pmCaffAmount'].fillna(0)
user_survey_2018_2['alcohol']=user_survey_2018_2['alcohol'].fillna(0)
user_survey_2018_2['aAmount']=user_survey_2018_2['aAmount'].fillna(0)

In [300]:
user_survey_2018_2['caffeine']=''
user_survey_2018_2['cAmount(ml)']=''
user_survey_2018_2['aAmount(ml)']=''

In [301]:
user_survey_2018_2['amCaffeine']=user_survey_2018_2['amCaffeine'].apply(lambda x: 0 if x==0 else 1)
user_survey_2018_2['pmCaffeine']=user_survey_2018_2['pmCaffeine'].apply(lambda x: 0 if x==0 else 1)
user_survey_2018_2['alcohol']=user_survey_2018_2['alcohol'].apply(lambda x: 0 if x==0 else 1)

In [302]:
user_survey_2018_2['caffeine']=user_survey_2018_2['amCaffeine']+user_survey_2018_2['pmCaffeine']

In [303]:
user_survey_2018_2['caffeine']=user_survey_2018_2['caffeine'].apply(lambda x: 0 if x==0 else 1)

In [304]:
user_survey_2018_2['cAmount(ml)']=user_survey_2018_2['amCaffAmount']+user_survey_2018_2['pmCaffAmount']

In [305]:
user_survey_2018_2['aAmount(ml)']=user_survey_2018_2['aAmount']

In [306]:
user_survey_2018_2=user_survey_2018_2[['date1','userId', 'pmEmotion', 'pmStress','caffeine',
       'cAmount(ml)', 'alcohol', 'aAmount(ml)']]

In [307]:
user2=user2.drop(columns='date1')

In [308]:
user2 = pd.merge(user2, user_survey_2018_2, left_on =['userId','date'], right_on=['userId', 'date1'], how = 'left' )

In [309]:
user_sleep_2019_2018_sorted_e4=user_sleep_2019_2018_sorted_e4[['userId','startDt','e4Bvp','e4Eda','e4Hr','e4Temp']]

In [310]:
user_sleep_2018_e4=user_sleep_2019_2018_sorted_e4.loc[user_sleep_2019_2018_sorted_e4['userId']<100,:]

In [311]:
user_sleep_2018_e4=user_sleep_2018_e4.reset_index(drop=True)

In [312]:
user_sleep_2018_e4['userId1']=""
for i in range(len(user_sleep_2018_e4)):
    if user_sleep_2018_e4['userId'][i]==1:
        user_sleep_2018_e4['userId1'][i]='user01'
    elif user_sleep_2018_e4['userId'][i]==2:
        user_sleep_2018_e4['userId1'][i]='user02'
    elif user_sleep_2018_e4['userId'][i]==3:
        user_sleep_2018_e4['userId1'][i]='user03'
    elif user_sleep_2018_e4['userId'][i]==4:
        user_sleep_2018_e4['userId1'][i]='user04'
    elif user_sleep_2018_e4['userId'][i]==5:
        user_sleep_2018_e4['userId1'][i]='user05'
    elif user_sleep_2018_e4['userId'][i]==6:
        user_sleep_2018_e4['userId1'][i]='user06'
    elif user_sleep_2018_e4['userId'][i]==7:
        user_sleep_2018_e4['userId1'][i]='user07'
    elif user_sleep_2018_e4['userId'][i]==8:
        user_sleep_2018_e4['userId1'][i]='user08'
    elif user_sleep_2018_e4['userId'][i]==9:
        user_sleep_2018_e4['userId1'][i]='user09'
    elif user_sleep_2018_e4['userId'][i]==10:
        user_sleep_2018_e4['userId1'][i]='user10'
    elif user_sleep_2018_e4['userId'][i]==11:
        user_sleep_2018_e4['userId1'][i]='user11'
    elif user_sleep_2018_e4['userId'][i]==12:
        user_sleep_2018_e4['userId1'][i]='user12'
    elif user_sleep_2018_e4['userId'][i]==13:
        user_sleep_2018_e4['userId1'][i]='user13'
    elif user_sleep_2018_e4['userId'][i]==14:
        user_sleep_2018_e4['userId1'][i]='user14'
    elif user_sleep_2018_e4['userId'][i]==15:
        user_sleep_2018_e4['userId1'][i]='user15'
    elif user_sleep_2018_e4['userId'][i]==16:
        user_sleep_2018_e4['userId1'][i]='user16'
    elif user_sleep_2018_e4['userId'][i]==17:
        user_sleep_2018_e4['userId1'][i]='user17'
    elif user_sleep_2018_e4['userId'][i]==18:
        user_sleep_2018_e4['userId1'][i]='user18'
    elif user_sleep_2018_e4['userId'][i]==19:
        user_sleep_2018_e4['userId1'][i]='user19'
    elif user_sleep_2018_e4['userId'][i]==20:
        user_sleep_2018_e4['userId1'][i]='user20'
    elif user_sleep_2018_e4['userId'][i]==21:
        user_sleep_2018_e4['userId1'][i]='user21'
    elif user_sleep_2018_e4['userId'][i]==22:
        user_sleep_2018_e4['userId1'][i]='user22'
    elif user_sleep_2018_e4['userId'][i]==23:
        user_sleep_2018_e4['userId1'][i]='user23'
    elif user_sleep_2018_e4['userId'][i]==24:
        user_sleep_2018_e4['userId1'][i]='user24'
    elif user_sleep_2018_e4['userId'][i]==25:
        user_sleep_2018_e4['userId1'][i]='user25'
    elif user_sleep_2018_e4['userId'][i]==26:
        user_sleep_2018_e4['userId1'][i]='user26'
    elif user_sleep_2018_e4['userId'][i]==27:
        user_sleep_2018_e4['userId1'][i]='user27'
    elif user_sleep_2018_e4['userId'][i]==28:
        user_sleep_2018_e4['userId1'][i]='user28'
    elif user_sleep_2018_e4['userId'][i]==29:
        user_sleep_2018_e4['userId1'][i]='user29'
    elif user_sleep_2018_e4['userId'][i]==30:
        user_sleep_2018_e4['userId1'][i]='user30'

<ipython-input-312-5cb09d308db8>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_sleep_2018_e4['userId1'][i]='user01'
<ipython-input-312-5cb09d308db8>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_sleep_2018_e4['userId1'][i]='user02'
<ipython-input-312-5cb09d308db8>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_sleep_2018_e4['userId1'][i]='user03'
<ipython-input-312-5cb09d308db8>:10: SettingWithCopyWarning: 
A val

In [313]:
user_sleep_2018_e4=user_sleep_2018_e4.drop(columns=['userId'])
user_sleep_2018_e4=user_sleep_2018_e4.rename(columns={'userId1':'userId'})

In [314]:
user_sleep_2018_e4=user_sleep_2018_e4.rename(columns={'startDt':'date'})

In [315]:
user_sleep_2018_e4

,date,e4Bvp,e4Eda,e4Hr,e4Temp,userId
0,2018-11-16 0:53,-0.005642,0.121614,80.006229,32.864825,user01
1,2018-11-17 3:36,0.006961,0.330542,72.347900,33.956935,user01
2,2018-11-18 5:06,-0.015563,0.290492,84.019970,33.884083,user01
3,2018-11-19 1:54,-0.001972,0.729625,80.477496,34.444794,user01
4,2018-11-20 1:38,0.005119,0.257681,80.851895,34.334449,user01
...,...,...,...,...,...,...
323,2018-11-24 3:32,0.000417,0.693704,84.231193,35.719695,user30
324,2018-11-25 1:29,-0.007869,0.430001,82.935526,35.455277,user30
325,2018-11-26 1:46,0.002362,0.157313,90.332692,34.672714,user30
326,2018-11-27 4:59,0.003062,1.362961,91.764735,35.568111,user30


In [316]:
user_sleep_2018_e4['date1']=''
for i in range(len(user_sleep_2018_e4)):
    user_sleep_2018_e4['date1'][i]=datetime.strptime(str(user_sleep_2018_e4['date'][i]),'%Y-%m-%d %H:%M').strftime('%Y-%m-%d')

<ipython-input-316-659c77a9e7e5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_sleep_2018_e4['date1'][i]=datetime.strptime(str(user_sleep_2018_e4['date'][i]),'%Y-%m-%d %H:%M').strftime('%Y-%m-%d')


In [317]:
user_sleep_2018_e4=user_sleep_2018_e4.drop(columns='date')

In [318]:
user2.columns

Index(['startDt', 'endDt', 'device', 'sleep_score', 'total_sleep_time',
       'time_in_bed', 'waso', 'wakeupcount', 'aal', 'movement_index',
       'fragmentation_index', 'sleep_frag_index', 'startdt1', 'enddt1',
       'count', 'date', 'userId', 'walk', 'exer', 'gps_activity_distance',
       'sleep', 'sleepProblem', 'dream', 'amCondition', 'amEmotion', 'date1',
       'pmEmotion', 'pmStress', 'caffeine', 'cAmount(ml)', 'alcohol',
       'aAmount(ml)'],
      dtype='object')

In [319]:
user2 = pd.merge(user2, user_sleep_2018_e4, left_on =['userId','date'], right_on=['userId', 'date1'], how = 'left' )

In [320]:
user_info_2018=user_info_2019_2018.iloc[20:,:]

In [321]:
user_info_2018=user_info_2018.drop(columns=['startDt','endDt'])

In [322]:
user2 = pd.merge(user2, user_info_2018, left_on =['userId'], right_on=['userId'], how = 'left' )

In [324]:
user2.columns

Index(['startDt', 'endDt', 'device', 'sleep_score', 'total_sleep_time',
       'time_in_bed', 'waso', 'wakeupcount', 'aal', 'movement_index',
       'fragmentation_index', 'sleep_frag_index', 'startdt1', 'enddt1',
       'count', 'date', 'userId', 'walk', 'exer', 'gps_activity_distance',
       'sleep', 'sleepProblem', 'dream', 'amCondition', 'amEmotion', 'date1_x',
       'pmEmotion', 'pmStress', 'caffeine', 'cAmount(ml)', 'alcohol',
       'aAmount(ml)', 'e4Bvp', 'e4Eda', 'e4Hr', 'e4Temp', 'date1_y', 'gender',
       'age', 'height', 'weight'],
      dtype='object')

In [327]:
user2=user2[['date','userId','walk', 'exer', 'sleep', 'sleepProblem','dream', 'amCondition', 'amEmotion',\
                        'pmEmotion', 'pmStress', 'caffeine','alcohol', 'gender', 'age',\
                       'height', 'weight','e4Bvp','e4Eda','e4Hr','e4Temp','gps_activity_distance']]

In [462]:
user1=user1[['date','userId','walk', 'exer', 'sleep', 'sleepProblem','dream', 'amCondition', 'amEmotion',\
                        'pmEmotion', 'pmStress', 'caffeine','alcohol', 'gender', 'age',\
                       'height', 'weight','e4Bvp','e4Eda','e4Hr','e4Temp','gps_activity_distance']]

In [396]:
user=user.rename(columns={'sleep_x':'sleep'})

In [397]:
user=user.rename(columns={'sleepProblem_x':'sleepProblem'})
user=user.rename(columns={'dream_x':'dream'})
user=user.rename(columns={'amCondition_x':'amCondition'})
user=user.rename(columns={'amEmotion_x':'amEmotion'})

In [398]:
user=user[['date','userId','walk', 'exer', 'sleep', 'sleepProblem','dream', 'amCondition', 'amEmotion',\
                        'pmEmotion', 'pmStress', 'caffeine','alcohol', 'gender', 'age',\
                       'height', 'weight','e4Bvp','e4Eda','e4Hr','e4Temp','gps_activity_distance']]

# 3. 2018-2020년 데이터 확인

## 3-1. 데이터 피처명 통일

In [464]:
features=user[['date','userId','walk', 'exer', 'sleep', 'sleepProblem','dream', 'amCondition', 'amEmotion',\
                        'pmEmotion', 'pmStress', 'caffeine', 'alcohol', 'gender', 'age',\
                       'e4Bvp','e4Eda','e4Hr','e4Temp','gps_activity_distance']]

In [465]:
features1=user1[['date','userId','walk', 'exer', 'sleep', 'sleepProblem','dream', 'amCondition', 'amEmotion',\
                        'pmEmotion', 'pmStress', 'caffeine','alcohol', 'gender', 'age',\
                       'e4Bvp','e4Eda','e4Hr','e4Temp','gps_activity_distance']]

In [466]:
features2=user2[['date','userId','walk', 'exer', 'sleep', 'sleepProblem','dream', 'amCondition', 'amEmotion',\
                        'pmEmotion', 'pmStress', 'caffeine', 'alcohol', 'gender', 'age',\
                       'e4Bvp','e4Eda','e4Hr','e4Temp','gps_activity_distance']]

## 3-2. 데이터 NULL 값 처리

>카페인, 알코올 -> NULL 값:0

In [467]:
features['caffeine']=features['caffeine'].fillna(0)
features['alcohol']=features['alcohol'].fillna(0)

<ipython-input-467-d3e2964be4a2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['caffeine']=features['caffeine'].fillna(0)
<ipython-input-467-d3e2964be4a2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['alcohol']=features['alcohol'].fillna(0)


>카페인, 알코올 -> YES or NO 로 변수 전처리

In [468]:
features['caffeine']=features['caffeine'].apply(lambda x: 0 if x==0 else 1)
features['alcohol']=features['alcohol'].apply(lambda x: 0 if x==0 else 1)

<ipython-input-468-32c2ed22b2f8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['caffeine']=features['caffeine'].apply(lambda x: 0 if x==0 else 1)
<ipython-input-468-32c2ed22b2f8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['alcohol']=features['alcohol'].apply(lambda x: 0 if x==0 else 1)


>운동 거리 -> NULL 값:0

In [469]:
features['gps_activity_distance']=features['gps_activity_distance'].fillna(0)

<ipython-input-469-5b8db13e3587>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['gps_activity_distance']=features['gps_activity_distance'].fillna(0)


>데이터 전처리는 위와 동일

In [470]:
features1['caffeine']=features1['caffeine'].fillna(0)
features1['alcohol']=features1['alcohol'].fillna(0)
features1['caffeine']=features1['caffeine'].apply(lambda x: 0 if x==0 else 1)
features1['alcohol']=features1['alcohol'].apply(lambda x: 0 if x==0 else 1)
features1['gps_activity_distance']=features1['gps_activity_distance'].fillna(0)

<ipython-input-470-619ad4f5cd3f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features1['caffeine']=features1['caffeine'].fillna(0)
<ipython-input-470-619ad4f5cd3f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features1['alcohol']=features1['alcohol'].fillna(0)
<ipython-input-470-619ad4f5cd3f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

In [471]:
features2['caffeine']=features2['caffeine'].fillna(0)
features2['alcohol']=features2['alcohol'].fillna(0)
features2['caffeine']=features2['caffeine'].apply(lambda x: 0 if x==0 else 1)
features2['alcohol']=features2['alcohol'].apply(lambda x: 0 if x==0 else 1)
features2['gps_activity_distance']=features2['gps_activity_distance'].fillna(0)

<ipython-input-471-4d6f4c2a8c69>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features2['caffeine']=features2['caffeine'].fillna(0)
<ipython-input-471-4d6f4c2a8c69>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features2['alcohol']=features2['alcohol'].fillna(0)
<ipython-input-471-4d6f4c2a8c69>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

In [472]:
features2['gender'].value_counts()

M    193
F    155
Name: gender, dtype: int64

>성별 전처리

In [473]:
features['gender']=features['gender'].apply(lambda x: 1 if x=='M' else 0)
features1['gender']=features1['gender'].apply(lambda x: 1 if x=='M' else 0)
features2['gender']=features2['gender'].apply(lambda x: 1 if x=='M' else 0)

<ipython-input-473-f9993d6ae814>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['gender']=features['gender'].apply(lambda x: 1 if x=='M' else 0)
<ipython-input-473-f9993d6ae814>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features1['gender']=features1['gender'].apply(lambda x: 1 if x=='M' else 0)
<ipython-input-473-f9993d6ae814>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cavea

In [474]:
features

,date,userId,walk,exer,sleep,sleepProblem,dream,amCondition,amEmotion,pmEmotion,pmStress,caffeine,alcohol,gender,age,e4Bvp,e4Eda,e4Hr,e4Temp,gps_activity_distance
0,2020-08-31,user01,0,20,3.0,6.0,2.0,3.0,3.0,4.0,3.0,1,0,1,27,NaN,NaN,NaN,NaN,0.0
1,2020-09-01,user01,0,0,2.0,3.0,2.0,2.0,3.0,4.0,3.0,1,0,1,27,NaN,NaN,NaN,NaN,0.0
2,2020-09-06,user01,80,0,3.0,7.0,1.0,3.0,3.0,4.0,4.0,1,0,1,27,NaN,NaN,NaN,NaN,0.0
3,2020-09-08,user01,45,0,4.0,3.0,2.0,3.0,4.0,4.0,4.0,1,1,1,27,-0.013653,3.103176,86.727616,33.021747,0.0
4,2020-09-09,user01,30,0,2.0,7.0,1.0,3.0,3.0,5.0,4.0,1,1,1,27,0.056283,2.503302,87.789920,34.441982,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
524,2020-09-23,user30,77,0,3.0,0.0,4.0,2.0,3.0,3.0,3.0,0,0,1,26,0.079167,0.160334,80.474611,31.358667,0.0
525,2020-09-24,user30,63,0,3.0,0.0,4.0,3.0,2.0,3.0,3.0,0,0,1,26,-0.137140,0.076865,73.867833,33.963111,0.0
526,2020-09-24,user30,63,0,3.0,0.0,4.0,3.0,2.0,3.0,3.0,0,0,1,26,-0.137140,0.076865,73.867833,33.963111,0.0
527,2020-09-25,user30,68,0,4.0,0.0,4.0,2.0,2.0,3.0,4.0,0,0,1,26,0.018909,0.159650,79.769218,33.107910,0.0


In [476]:
features=features.dropna(axis=0)
features1=features1.dropna(axis=0)
features2=features2.dropna(axis=0)

In [633]:
features['year']=2020
features1['year']=2019
features2['year']=2018

<ipython-input-633-12dc1f164a08>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['year']=2020


>각 연도별 데이터 csv 저장

In [634]:
features.to_csv('user0.csv')
features1.to_csv('user1.csv')
features2.to_csv('user2.csv')

In [635]:
features

,date,userId,walk,exer,sleep,sleepProblem,dream,amCondition,amEmotion,pmEmotion,...,caffeine,alcohol,gender,age,e4Bvp,e4Eda,e4Hr,e4Temp,gps_activity_distance,year
3,2020-09-08,user01,45,0,4.0,3.0,2.0,3.0,4.0,4.0,...,1,1,1,27,-0.013653,3.103176,86.727616,33.021747,0.0,2020
4,2020-09-09,user01,30,0,2.0,7.0,1.0,3.0,3.0,5.0,...,1,1,1,27,0.056283,2.503302,87.789920,34.441982,0.0,2020
6,2020-09-13,user01,20,0,2.0,3.0,4.0,2.0,3.0,5.0,...,1,1,1,27,0.036912,3.373406,85.270147,33.193400,0.0,2020
7,2020-09-14,user01,20,0,2.0,3.0,2.0,3.0,3.0,3.0,...,0,0,1,27,0.009140,1.663482,89.475536,34.086428,0.0,2020
8,2020-09-15,user01,35,0,4.0,3.0,2.0,3.0,3.0,4.0,...,1,1,1,27,-0.004555,0.773490,88.103819,34.190778,0.0,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
524,2020-09-23,user30,77,0,3.0,0.0,4.0,2.0,3.0,3.0,...,0,0,1,26,0.079167,0.160334,80.474611,31.358667,0.0,2020
525,2020-09-24,user30,63,0,3.0,0.0,4.0,3.0,2.0,3.0,...,0,0,1,26,-0.137140,0.076865,73.867833,33.963111,0.0,2020
526,2020-09-24,user30,63,0,3.0,0.0,4.0,3.0,2.0,3.0,...,0,0,1,26,-0.137140,0.076865,73.867833,33.963111,0.0,2020
527,2020-09-25,user30,68,0,4.0,0.0,4.0,2.0,2.0,3.0,...,0,0,1,26,0.018909,0.159650,79.769218,33.107910,0.0,2020


>연도별 데이터 병합 및 csv 저장

In [636]:
final=pd.concat([features,features1,features2])

In [4]:
final=pd.read_csv('final.csv')